In [180]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

COVID = pd.read_excel('./owid-covid-data-ko.xlsx')
COVID = COVID.fillna(0)
COVID.head()
date = pd.DataFrame({'일자':COVID['date']})

In [181]:
COVID = COVID[['new_cases', 'new_deaths', 'icu_patients',
       'total_vaccinations', 'total_boosters', 'new_vaccinations',
       'stringency_index']]

In [182]:
# COVID.drop(['iso_code','continent','location','date','tests_units',], axis=1,inplace=True)
COVID.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 838 entries, 0 to 837
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   new_cases           838 non-null    float64
 1   new_deaths          838 non-null    float64
 2   icu_patients        838 non-null    float64
 3   total_vaccinations  838 non-null    float64
 4   total_boosters      838 non-null    float64
 5   new_vaccinations    838 non-null    float64
 6   stringency_index    838 non-null    float64
dtypes: float64(7)
memory usage: 46.0 KB


In [183]:
X = COVID.drop(['new_deaths'], axis=1)
y = COVID['new_deaths']

In [184]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13)

In [185]:
from sklearn.linear_model import LinearRegression

reg = LinearRegression()
reg.fit(X_train, y_train)

LinearRegression()

In [186]:
from sklearn.metrics import mean_squared_error

pred_tr = reg.predict(X_train)
pred_test = reg.predict(X_test)

rmse_tr = np.sqrt(mean_squared_error(y_train, pred_tr))
rmse_test = np.sqrt(mean_squared_error(y_test, pred_test))

In [187]:
print('RMSE of Train Data: ', rmse_tr)
print('RMSE of Test Data: ', rmse_test)

RMSE of Train Data:  22.79200827645837
RMSE of Test Data:  22.603950455288746


In [188]:
lm = sm.OLS(y, X).fit()

In [189]:
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:             new_deaths   R-squared (uncentered):                   0.904
Model:                            OLS   Adj. R-squared (uncentered):              0.903
Method:                 Least Squares   F-statistic:                              1305.
Date:                Tue, 10 May 2022   Prob (F-statistic):                        0.00
Time:                        15:50:53   Log-Likelihood:                         -3808.6
No. Observations:                 838   AIC:                                      7629.
Df Residuals:                     832   BIC:                                      7658.
Df Model:                           6                                                  
Covariance Type:            nonrobust                                                  
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
new_cases              0.0005   1.83e-05     28.831      0.000       0.000       0.001
icu_patients           0.1112      0.006     20.108      0.000       0.100       0.122
total_vaccinations -3.682e-07   5.18e-08     -7.104      0.000    -4.7e-07   -2.66e-07
total_boosters      1.303e-06   1.85e-07      7.047      0.000     9.4e-07    1.67e-06
new_vaccinations   -1.327e-05   3.89e-06     -3.409      0.001   -2.09e-05   -5.63e-06
stringency_index      -0.1337      0.020     -6.627      0.000      -0.173      -0.094
==============================================================================
Omnibus:                      312.885   Durbin-Watson:                   0.939
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            10066.210
Skew:                           1.036   Prob(JB):                         0.00
Kurtosis:                      19.852   Cond. No.                     1.45e+06
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[3] The condition number is large, 1.45e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [190]:
p = lm.predict(X).round(0)
p_dead = pd.DataFrame({'추정사망(명)':p})
p_dead[p_dead['추정사망(명)']<=0] = 0

In [199]:
predict_COVID = pd.concat([date,COVID, p_dead], axis=1)
predict_COVID.tail(10)

,일자,new_cases,new_deaths,icu_patients,total_vaccinations,total_boosters,new_vaccinations,stringency_index,추정사망(명)
828,2022-04-29,43262.0,70.0,526.0,123228407.0,35164493.0,394958.0,16.67,74.0
829,2022-04-30,37771.0,81.0,490.0,123311945.0,35245641.0,83538.0,16.67,72.0
830,2022-05-01,20084.0,83.0,493.0,123313126.0,35246797.0,1181.0,16.67,64.0
831,2022-05-02,51020.0,49.0,461.0,123480197.0,35412071.0,167071.0,0.00,77.0
832,2022-05-03,49038.0,72.0,417.0,123661521.0,35591680.0,181324.0,0.00,71.0
833,2022-05-04,42277.0,79.0,432.0,123823522.0,35751825.0,162001.0,0.00,69.0
834,2022-05-05,26714.0,48.0,441.0,123827158.0,35755365.0,3636.0,0.00,64.0
835,2022-05-06,39552.0,83.0,423.0,124081608.0,36005262.0,254450.0,0.00,66.0
836,2022-05-07,40064.0,71.0,419.0,124131394.0,36053704.0,49786.0,0.00,68.0
837,2022-05-08,20601.0,40.0,423.0,0.0,0.0,0.0,0.00,58.0


In [202]:
predict_COVID = predict_COVID[['일자','new_cases','icu_patients','total_vaccinations','total_boosters','new_vaccinations','stringency_index','new_deaths','추정사망(명)']]
predict_COVID.drop(837,axis=0,inplace=True)
predict_COVID.set_index('일자', inplace=True)
predict_COVID.tail()

,new_cases,icu_patients,total_vaccinations,total_boosters,new_vaccinations,stringency_index,new_deaths,추정사망(명)
일자,,,,,,,,
2022-05-03,49038.0,417.0,123661521.0,35591680.0,181324.0,0.0,72.0,71.0
2022-05-04,42277.0,432.0,123823522.0,35751825.0,162001.0,0.0,79.0,69.0
2022-05-05,26714.0,441.0,123827158.0,35755365.0,3636.0,0.0,48.0,64.0
2022-05-06,39552.0,423.0,124081608.0,36005262.0,254450.0,0.0,83.0,66.0
2022-05-07,40064.0,419.0,124131394.0,36053704.0,49786.0,0.0,71.0,68.0


In [204]:
predict_COVID.to_excel('./predict_COVID.xlsx',encoding='utf-8-sig')